In [1]:
import dash
from dash import dcc, html
import pandas as pd
import plotly.graph_objs as go
from dash.dependencies import Input, Output
import plotly.express as px


#incializo aplicación
app = dash.Dash(__name__)

#cargo df y concateno
df_1 = pd.read_csv('parte_1.csv')
df_2 = pd.read_csv('parte_2.csv')
df = pd.concat([df_1, df_2])




/var/folders/p4/277gn6c95d756wbt3sxs8ch80000gn/T/ipykernel_8214/1006916710.py:13: DtypeWarning:

Columns (7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.

/var/folders/p4/277gn6c95d756wbt3sxs8ch80000gn/T/ipykernel_8214/1006916710.py:14: DtypeWarning:

Columns (7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.



In [2]:
# NIVEL 1 

# 1.1) Calculamos las métricas básicas usando nunique
num_tiendas = df['store_nbr'].nunique()
num_productos = df['family'].nunique()
num_estados = df['state'].nunique()


# 1.2) Calculamos los 10 productos más vendidos 

#Usamos groupby para agrupar por el tipo de prdoucto y luego aplicamos las metricas sobre sales creando un nuevo df 

top_10_productos = (df.groupby('family')['sales']
                 .sum()
                 .sort_values(ascending=False)
                 .head(10) #head 10 para solo los top 10
                 .reset_index())


# Creamos el gráfico de barras usando ploty y trabajamos con el datraframe top_10_productos creado anteriormente
fig_barras = go.Figure(
    data=[
        go.Bar(
            x=top_10_productos['family'],
            y=top_10_productos['sales'],
            marker=dict(color='blue'),
        )
    ],
    layout=go.Layout(
        title="Top 10 productos más vendidos",
        xaxis=dict(title="Productos"),
        yaxis=dict(title="Ventas"),
    )
)


#1.3) Calculamos las ventas mensuales

#primero creamos el nuevo df ventas mensuales con las columnas year, month y sales 
ventas_por_mes = (df.groupby(['year', 'month'])['sales']
                    .sum()
                    .reset_index())
#creamos una nueva columna fecha utilizanfo la funcion dxe pandas to_datetime 
ventas_por_mes['fecha'] = pd.to_datetime(ventas_por_mes[['year', 'month']].assign(day=1))


# creamos grafico de lineas con dataframe creado ventas_por_mes 
# dibujamos los datso con go.Scatter (puntos) y los conectamos con lineas con el modo mode = "lines+markers"
fig_lineas = go.Figure(
    data=[
        go.Scatter(
            x=ventas_por_mes['fecha'],
            y=ventas_por_mes['sales'],
            mode='lines+markers', #markers simplemente añade los puntos encima de la linea para que se vea claramente cada uno de los datos
            line=dict(color='green'),
            name="Ventas Mensuales"
        )
    ],
    layout=go.Layout(
        title="Ventas Mensuales",
        xaxis=dict(title="Fecha"),
        yaxis=dict(title="Ventas"),
    )
)

 


/Users/javiermendoza/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [3]:
#NIVEL 2 

#Utlizamos los componentes de entarda y salida para hacer el dashboard interactivo ( el usuario elige la tienda)

@app.callback(
    Output('ventas-tienda', 'figure'),  # Actualiza el gráfico
    Input('dropdown-tienda', 'value')  # Escucha la tienda seleccionada
)
def actualiza_ventas(tienda_seleccionada):
    if tienda_seleccionada is None:
        return go.Figure()  # devuelve un gráfico vacío si no se selecciona tienda

    # creamos nuevo df filtrando los datos por la tienda selccionada
    datos_filtrados = df[df['store_nbr'] == tienda_seleccionada]
    #creamos nuevo df siguiendo la misma dinamica que en el nivel 1 para las ventas mensuales (groupby month y year y operamos sobre sales) solo que ahora sobre el df filtrado solo para el año que nos interesa
    ventas_anuales = datos_filtrados.groupby(['year', 'month'])['sales'].sum().reset_index()
    ventas_anuales['fecha'] = pd.to_datetime(ventas_anuales[['year', 'month']].assign(day=1))

    #de nuevo usamos plotly para grafico de lineas similar al anterior 
    fig = go.Figure(
        data=[
            go.Scatter(
                x=ventas_anuales['fecha'],
                y=ventas_anuales['sales'],
                mode='lines+markers', 
                line=dict(color='blue'),
                name=f"Ventas Tienda {tienda_seleccionada}"
            )
        ],
        layout=go.Layout(
            title=f"Ventas Anuales - Tienda {tienda_seleccionada}",
            xaxis=dict(title="Fecha"),
            yaxis=dict(title="Ventas"),
        )
    )
    return fig

@app.callback(
    Output('tabla-promociones', 'children'),  #actualiza la tabla 
    Input('dropdown-tienda', 'value')  #escucha la tienda seleccionada
)

def actualiza_tabla(tienda_seleccionada):
    if tienda_seleccionada is None:
        return html.Div([
            html.P("Por favor, selecciona una tienda para ver los productos en promoción.")
        ])

    # creamos df promciones incluyendo solo las filas para la tienda selccionada que además tiene elemento en la columna "onprimotion" distinto de 0 
    promociones = df[(df['store_nbr'] == tienda_seleccionada) & (df['onpromotion'] > 0)]

    if promociones.empty: #manejamos el caso en el que no hay ninguna promocion 
        return html.Div([
            html.H3("No hay productos en promoción para esta tienda."),
        ])

    # he interpretado los datos como que las promociones de los productos de la misma familia son iguales por eso he hecho un groupby en "family" y he sumado los que estaban en promocion 
    promociones_agrupadas = (promociones.groupby('family', as_index=False)
                             .agg({'onpromotion': 'sum'}))

    # como estadistica general se proporciona al ususario el total de productos en promocion 
    productos_distintos = promociones_agrupadas['family'].nunique()

    #creamos la tabla con html.Table 
    #para mejorar el diseño hemos añadido elementos de decoracion personalizacion inspirandome en alguno de los encontrados en el archivo  dash_styling.ipynb
    return html.Div([
        html.H3(f"Total de productos distintos en promoción: {productos_distintos}",
                style={'color': 'green', 'marginBottom': '20px'}),
        html.Table(
            #creamos el encabezado de la tabla con html.Thead 
            [html.Thead(html.Tr([
                html.Th("Producto", style={'backgroundColor': '#4CAF50', 'color': 'white', 'padding': '10px'}),
                html.Th("Promociones", style={'backgroundColor': '#4CAF50', 'color': 'white', 'padding': '10px'})
            ]))] +
            #creamos el cuerpo de la tabla con html.Tbody 
            [html.Tbody([
                html.Tr([
                    html.Td(row['family'], style={'border': '1px solid #ddd', 'padding': '8px'}),
                    html.Td(row['onpromotion'], style={'border': '1px solid #ddd', 'padding': '8px'}) #padding crea espacio interno para que se vea mas claro 
                ], style={'backgroundColor': '#f9f9f9' if i % 2 == 0 else 'white'}) #añadaimos color de fondo en la tabla 
                for i, row in promociones_agrupadas.iterrows()
            ])],
            #unificamos los bordes, añadimos margen superior y alineamso el contenido 
            style={
                'width': '100%',
                'borderCollapse': 'collapse',
                'marginTop': '20px',
                'textAlign': 'left'
            }
        )
    ])




In [4]:
#NIVEL 3
 
@app.callback(
    Output('grafico-estacionalidad', 'figure'),
    [Input('filtro-ano', 'value'),
     Input('filtro-estado', 'value')]
)

#3.1) Análisis de estacionalidad 
def actualizar_grafico_estacionalidad(ano, estado): #filtramos segun el año segun decide el usuario 
    
    #creamos nuevamente df con los datos filtrados 
    datos_filtrados = df
    if ano is not None:
        datos_filtrados = datos_filtrados[datos_filtrados['year'] == ano]
    if estado is not None:
        datos_filtrados = datos_filtrados[datos_filtrados['state'] == estado]

    #inicializo objeto figure 
    fig = go.Figure()

    #obtenemos df de las ventas diarias con groupby para la fecha exacta y aplicamos la operacion sum sobre las ventas 
    ventas_diarias = datos_filtrados.groupby('date')['sales'].sum().reset_index()
    #añadamis scatter con los datos del df 
    fig.add_trace(go.Scatter(
        x=ventas_diarias['date'],
        y=ventas_diarias['sales'],
        mode='lines',
        name='Ventas diarias'
    ))

    # obtenemos df de promociones diarias de la misma manera ahora con la columna onpromotion 
    promociones_diarias = datos_filtrados.groupby('date')['onpromotion'].sum().reset_index()
    fig.add_trace(go.Scatter(
        x=promociones_diarias['date'],
        y=promociones_diarias['onpromotion'],
        mode='lines',
        line=dict(color='green'), 
        name='Promociones diarias'
    ))

    #los festivas son los que no son nan en la columna de holiday_type 
    dias_festivos = datos_filtrados[pd.notna(datos_filtrados['holiday_type'])] 
    ventas_festivos = ventas_diarias[ventas_diarias['date'].isin(dias_festivos['date'])] #aqui nos aseguramos que el punto de festivos aparezca encima de la venta 
    fig.add_trace(go.Scatter(
        x=ventas_festivos['date'],
        y=ventas_festivos['sales'],   
        mode='markers',
        marker=dict(size=5, color='red'),  
        name='Días Festivos'
    ))

    #Diseño del gráficp (ejes y leyenda)
    fig.update_layout(
        title="Análisis de Estacionalidad",
        xaxis_title="Fecha",
        yaxis_title="Valores",
        legend_title="Variables"
    )

    return fig

#3.2) Análisis comparativo multidimensional 
@app.callback(
    Output('grafico-burbujas', 'figure'),
    [Input('filtro-ano', 'value'),
     Input('filtro-estado', 'value'),
     Input('agrupamiento', 'value')]
)
def actualizar_grafico_burbujas(ano, estado, agrupamiento):
    #creo df filtrado
    datos_filtrados = df.copy()
    if ano is not None:
        datos_filtrados = datos_filtrados[datos_filtrados['year'] == ano]
    if estado is not None:
        datos_filtrados = datos_filtrados[datos_filtrados['state'] == estado]

    #elimino nulos
    datos_filtrados = datos_filtrados.dropna(subset=['sales', 'onpromotion', 'transactions'])

    #crep columna de ventas medias agrupando por tienda y fecha 
    datos_filtrados['avg_sales'] = datos_filtrados.groupby(['store_nbr', 'date'])['sales'].transform('mean')

    #calculao el porcentaje de prod en promoción 
    datos_filtrados['promotion_pct'] = datos_filtrados.apply(
        lambda row: (row['onpromotion'] / row['sales'] * 100) if row['sales'] > 0 else 0, #else 0 para evitar diivisonError
        axis=1
    )

    #transacciones para tamaño de burbuja 
    datos_filtrados['transactions'] = pd.to_numeric(datos_filtrados['transactions'], errors='coerce')
    datos_filtrados['transactions'].fillna(0, inplace=True)  # Reemplazar nulos con 0
    datos_filtrados = datos_filtrados[datos_filtrados['transactions'] >= 0]

    #creo gráfico de burbujas con scatter 
    fig = px.scatter(
        datos_filtrados,
        x='avg_sales',  # Ventas promedio por tienda
        y='promotion_pct',  # Porcentaje de productos en promoción
        size='transactions',  # Tamaño de la burbuja: número de transacciones
        color=agrupamiento or 'store_type',  # Agrupamiento
        animation_frame='date',  # Animación por fechas
        hover_data={
            'store_nbr': True,
            'avg_sales': ':.2f',
            'promotion_pct': ':.2f',
            'transactions': True
        },
        title="Gráfico de Burbujas Interactivo"
    )

    #diseño
    fig.update_layout(
        xaxis_title="Ventas Promedio por Tienda",
        yaxis_title="Porcentaje de Productos en Promoción",
        legend_title="Agrupamiento"
    )

    return fig



#3.3) Análisis profundo de patrones de venta

#añadimos un input mas (filtro) a los que ya habia en 3.1 y 3.2, este filtro es el tipo de producto 
@app.callback(
    Output('heatmap-patrones', 'figure'),
    [Input('filtro-familia', 'value'),
     Input('tipo-normalizacion', 'value'),
     Input('filtro-ano', 'value'),
     Input('filtro-estado', 'value')]   
)
def actualizar_heatmap(familia, normalizacion, ano, estado):
    
    df['date'] = pd.to_datetime(df['date'], errors='coerce') #aplicamos pd.datetime a a columna date del df orginal 

    #eliminamos los nulos y filtramos segun los inputs 
    datos_filtrados = df[df['date'].notna()]

    if familia:
        datos_filtrados = datos_filtrados[datos_filtrados['family'] == familia]
    if ano:
        datos_filtrados = datos_filtrados[datos_filtrados['year'] == ano]
    if estado:
        datos_filtrados = datos_filtrados[datos_filtrados['state'] == estado]

    #extraemos semana del año y día de la semana
    datos_filtrados['week_of_year'] = datos_filtrados['date'].dt.isocalendar().week
    datos_filtrados['day_of_week'] = datos_filtrados['date'].dt.day_name()

    #agrupamos los datos por semana del año y día de la semana
    ventas_agrupadas = datos_filtrados.groupby(['week_of_year', 'day_of_week'])['sales'].sum().reset_index()

    #normalización de los datos (dividimos por el total)
    if normalizacion == 'relativo':
        ventas_agrupadas['sales'] = ventas_agrupadas['sales'] / ventas_agrupadas['sales'].sum()

    #usamos density_heatmap 
    fig = px.density_heatmap(
        ventas_agrupadas,
        x='week_of_year',
        y='day_of_week',
        z='sales',
        color_continuous_scale='Viridis',
        title="Patrones de Venta por Semana y Día",
        labels={'week_of_year': 'Semana del Año', 'day_of_week': 'Día de la Semana', 'sales': 'Ventas'}
    )

    return fig

 

In [5]:


# LAYOUT
""" 
Para el Layout vamos a usar 3 pestañas una para cada nivel, dentro de cada pestaña se desarollan las funcionalidades pedidas 
"""
app.layout = html.Div([
    dcc.Tabs([
        #Pestaña para el Nivel 1
        dcc.Tab(label='N1: Resumen de Métricas Básicas', children=[
            html.Div([
                html.H1("Dashboard - Nivel 1"),
                html.Div(id='metricas-basicas', children=[
                    html.H3(f"Número total de tiendas: {num_tiendas}",
                            style={'color': 'blue'}),
                    html.H3(f"Número total de productos: {num_productos}",
                            style={'color': 'blue'}),
                    html.H3(f"Total de estados: {num_estados}",
                            style={'color': 'blue'})
                ]),
                dcc.Graph(
                    id='grafico-barras',
                    figure=fig_barras
                ),
                dcc.Graph(
                    id='grafico-lineas',
                    figure=fig_lineas
                ),
            ])
        ]),
        #Pestaña para el Nivel 2
        dcc.Tab(label= "N2: Ventas por Tienda", children=[
            html.Div([
                html.H1("Dashboard - Nivel 2"),
                dcc.Dropdown(
                    id='dropdown-tienda',
                    options=[
                        {'label': f'Tienda {tienda}', 'value': tienda}
                        for tienda in sorted(df['store_nbr'].unique())
                    ],
                    placeholder="Selecciona una tienda",
                    style={'width': '50%'}
                ),
                dcc.Graph(id='ventas-tienda'),  # Gráfico dinámico
                html.Div(id='tabla-promociones')  # Tabla dinámica
            ])
        ]),
        #Pestaña para el Nivel 3
        dcc.Tab(label="N3: Análisis Avanzado", children=[
                html.Div([
                    html.H1("Dashboard - Nivel 3"),
                    # Filtros globales
                    dcc.Dropdown(
                        id='filtro-ano',
                        options=[{'label': str(ano), 'value': ano} for ano in sorted(df['year'].unique())],
                        placeholder="Selecciona un año",
                        style={'width': '50%', 'marginBottom': '20px'}
                    ),
                    dcc.Dropdown(
                        id='filtro-estado',
                        options=[{'label': estado, 'value': estado} for estado in sorted(df['state'].unique())],
                        placeholder="Selecciona un estado",
                        style={'width': '50%', 'marginBottom': '20px'}
                    ),
                    # Gráfico de estacionalidad
                    dcc.Graph(id='grafico-estacionalidad'),
                    html.Hr(),
                    # Gráfico de burbujas
                    html.H2("Análisis Comparativo Multidimensional"),
                    dcc.RadioItems(
                        id='agrupamiento',
                        options=[
                            {'label': 'Agrupar por Tipo de Tienda', 'value': 'store_type'},
                            {'label': 'Agrupar por Cluster', 'value': 'cluster'}
                        ],
                        value='store_type',
                        style={'marginBottom': '20px'}
                    ),
                    dcc.Graph(id='grafico-burbujas'),
                    html.Hr(),
                    # Heatmap de patrones de venta
                    html.H2("Análisis Profundo de Patrones de Venta"),
                    dcc.Dropdown(
                        id='filtro-familia',
                        options=[{'label': familia, 'value': familia} for familia in sorted(df['family'].unique())],
                        placeholder="Selecciona una familia de productos",
                        style={'width': '50%', 'marginBottom': '20px'}
                    ),
                    dcc.Dropdown(
                        id='filtro-estado',
                        options=[{'label': estado, 'value': estado} for estado in sorted(df['state'].unique())],
                        placeholder="Selecciona un estado",
                        style={'width': '50%', 'marginBottom': '20px'}
                    ),

                    dcc.RadioItems(
                        id='tipo-normalizacion',
                        options=[
                            {'label': 'Valores Absolutos', 'value': 'absoluto'},
                            {'label': 'Valores Relativos', 'value': 'relativo'}
                        ],
                        value='absoluto',
                        style={'marginBottom': '20px'}
                    ),
                    dcc.RadioItems(
                        id='comparacion-periodos',
                        options=[
                            {'label': 'Sin Comparación', 'value': 'sin_comparacion'},
                            {'label': 'Año Actual vs Anterior', 'value': 'vs_anterior'}
                        ],
                        value='sin_comparacion',
                        style={'marginBottom': '20px'}
                    ),
                    dcc.Graph(id='heatmap-patrones')
                ])
            ]),

        dcc.Tab(
            label='Documentación',
            children=[
                html.Div([
                    html.H2('Documentación de Decisiones de Diseño y Análisis'),
                    dcc.Markdown('''
                    ### Decisiones de Diseño:
                    - **Gráfico de barras(nivel 1)**: Se seleccionó para mostrar los 10 productos más vendidos porque facilita la comparación visual rápida de categorías discretas
                    - **Gráfico de líneas(nivel 1,2,3) **: Útil para visualizar tendencias temporales, como las ventas mensuales

                    ### Algunas decisiones de Análisis:
                    - Los datos de ventas se analizaron utilizando estadísticos descriptivos para identificar valores atípicos y rangos relevantes
                    - Se aplicaron técnicas de visualización para resaltar correlaciones entre las categorías de productos y sus ventas (ejemplo colores y tamaños varibles)


                    ### Justificación del Diseño:
                    - El uso de pestañas proporciona una navegación organizada para diferentes tipos de visualizaciones
                    - La estética simple facilita la interpretación rápida de los datos.
                    '''),
                ])
            ])
        ])
    ])

In [6]:
if __name__ == '__main__':
    app.run_server(debug=True, host='0.0.0.0', port=8080) #este es un puerto ejemplo para ejeuctra la url http://ip:8080



---------------------------------------------------------------------------
DuplicateIdError                          Traceback (most recent call last)
DuplicateIdError: Duplicate component id found in the initial layout: `filtro-estado`



/Users/javiermendoza/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/Users/javiermendoza/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/Users/javiermendoza/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.ar